# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [147]:
import numpy as np
import pandas as pd
import xlsxwriter 
from scipy import stats
import math
import requests

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [ ]:
stocks = pd.read_csv("sp_500_stocks.csv")
IEX_CLOUD_API_TOKEN = "Tpk_059b97af715d417d9f49f50b51b1c448"
stocks

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [31]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'avgTotalVolume': 115699520,
 'calculationPrice': 'close',
 'change': 1.15,
 'changePercent': 0.0088,
 'close': 132.36,
 'closeSource': 'lciafiof',
 'closeTime': 1665583822560,
 'companyName': 'Apple Inc',
 'delayedPrice': 136.56,
 'delayedPriceTime': 1624840542805,
 'extendedChange': -0.3,
 'extendedChangePercent': -0.0022,
 'extendedPrice': 134.6,
 'extendedPriceTime': 1630292197028,
 'high': 134.76,
 'highSource': 'eyiedce5pramint 1d elu ',
 'highTime': 1649968432389,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 137.6,
 'iexCloseTime': 1671457796453,
 'iexLastUpdated': 1679616569472,
 'iexMarketPercent': 0.013366758563301391,
 'iexOpen': 138.32,
 'iexOpenTime': 1637892799892,
 'iexRealtimePrice': 134.72,
 'iexRealtimeSize': 37,
 'iexVolume': 1364023,
 'isUSMarketOpen': False,
 'lastTradeTime': 1643135895327,
 'latestPrice': 137.96,
 'latestSource': 'Close',
 'latestTime': 'January 8, 2021',
 'latestUpdate': 1668013305115,
 'latestVolume': 1

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
symbol_strings

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
  batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
  data = requests.get(batch_api_url).json()
  for symbol in data:
    final_dataframe = final_dataframe.append(
        pd.Series(
            [ symbol,
            data[symbol]["quote"]["latestPrice"],
            data[symbol]["quote"]["peRatio"],
            "N/A"
            ],index = my_columns
        ),
      ignore_index = True
    )

final_dataframe

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [ ]:
final_dataframe.sort_values("Price-to-Earnings Ratio", inplace=True)
final_dataframe = final_dataframe[final_dataframe["Price-to-Earnings Ratio"] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop("index", axis = 1, inplace = True)
final_dataframe

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [56]:
def portfolio_input():
    global portfolio_size
    portfolio_size = float(input("Enter the value of your portfolio:"))

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [57]:
portfolio_input()


Enter the value of your portfolio:1000000



You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [ ]:
position_size = portfolio_size / len(final_dataframe.index)
for i in final_dataframe.index:
  final_dataframe.loc[i, ["Number of Shares to Buy"]] = math.floor(position_size/final_dataframe.loc[i, ["Price"]])

final_dataframe


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [84]:
symbol = "AAPL"
batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(batch_api_url).json()
# Price-to-earnings ratio
pe_ratio = data[symbol]["quote"]["peRatio"]
print(pe_ratio)
# Price-to-book ratio
pb_ratio = data[symbol]["advanced-stats"]["priceToBook"]
print(pb_ratio)
# Price-to-sales ratio
ps_ratio = data[symbol]["advanced-stats"]['priceToSales']
print(ps_ratio)
# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev_to_ebitda = data[symbol]["advanced-stats"]['enterpriseValue']/data[symbol]["advanced-stats"]["EBITDA"]
print(ev_to_ebitda)
# Enterprise Value divided by Gross Profit (EV/GP)
ev_to_gross_profit = data[symbol]["advanced-stats"]['enterpriseValue']/data[symbol]["advanced-stats"]["grossProfit"]
print(ev_to_gross_profit)
# data

40.85
35
8.11
28.028737424138825
20.823906861571054


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [ ]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
  batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
  data = requests.get(batch_api_url).json()
  for symbol in symbol_string.split(","):
    try:
      ev_to_ebitda = data[symbol]["advanced-stats"]['enterpriseValue']/data[symbol]["advanced-stats"]["EBITDA"]
    except TypeError:
      ev_to_ebitda = np.NaN

    try: 
      ev_to_gp = data[symbol]["advanced-stats"]['enterpriseValue']/data[symbol]["advanced-stats"]["grossProfit"]
    except TypeError:
      ev_to_gp = np.NaN
    
    rv_dataframe = rv_dataframe.append(
        pd.Series(
            [
              symbol,
              data[symbol]["quote"]["latestPrice"],
              'N/A', 
              data[symbol]["quote"]["peRatio"],
              'N/A',
              data[symbol]["advanced-stats"]["priceToBook"],
              'N/A',
              data[symbol]["advanced-stats"]['priceToSales'],
              'N/A',
              ev_to_ebitda,
              'N/A',
              ev_to_gp,
              'N/A',
              'N/A' 
            ],
            index = rv_columns),
        ignore_index = True 
    )

rv_dataframe


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [ ]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
  rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [138]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sales Ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile',
}

for metric in metrics.keys():
  for row in rv_dataframe.index:
    rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric],rv_dataframe.loc[row, metric])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.66,N/A,55.47,85.7426,8.00,73.4653,7.5000,80.9901,35.129122,84.7525,14.201000,82.3762,N/A
1,AAL,15.66,N/A,-1.11,14.4554,-1.68,4.55446,0.3813,2.17822,-6.165421,2.77228,1.401192,5.34653,N/A
2,AAP,175.71,N/A,25.37,47.1287,3.09,41.9802,1.2300,18.1188,11.499209,27.7228,2.652338,12.6733,N/A
3,AAPL,133.72,N/A,41.11,73.6634,34.00,96.0396,8.2800,84.1584,29.332645,81.7822,21.066187,93.8614,N/A
4,ABBV,110.39,N/A,23.56,44.7525,13.00,85.1485,4.8900,68.7129,16.218431,47.9208,9.445878,63.3663,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.53,N/A,32.55,62.1782,-4.13,4.15842,5.9500,74.3564,23.323840,71.4851,9.889702,66.1386,N/A
501,ZBH,165.08,N/A,-224.63,1.9802,2.87,38.5149,4.6800,63.5644,27.916793,79.2079,8.121371,50.8911,N/A
502,ZBRA,421.33,N/A,46.98,77.8218,11.30,83.3663,5.2100,70.396,33.068671,84.3564,11.978332,74.4554,N/A
503,ZION,49.80,N/A,20.80,40,1.16,11.5842,2.7300,44.4554,12.395598,32.8713,3.205843,17.2277,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [139]:
from statistics import mean

for row in rv_dataframe.index:
  PE_ratio = rv_dataframe.loc[row, "PE Percentile"]
  PB_ratio = rv_dataframe.loc[row, "PB Percentile"]
  PS_ratio = rv_dataframe.loc[row, "PS Percentile"]
  EV_EBITDA = rv_dataframe.loc[row, "EV/EBITDA Percentile"]
  EV_GP = rv_dataframe.loc[row, "EV/GP Percentile"]
  rv_dataframe.loc[row, "RV Score"] = mean([PE_ratio,PB_ratio,PS_ratio,EV_EBITDA,EV_GP])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.66,N/A,55.47,85.7426,8.00,73.4653,7.5000,80.9901,35.129122,84.7525,14.201000,82.3762,81.4653
1,AAL,15.66,N/A,-1.11,14.4554,-1.68,4.55446,0.3813,2.17822,-6.165421,2.77228,1.401192,5.34653,5.86139
2,AAP,175.71,N/A,25.37,47.1287,3.09,41.9802,1.2300,18.1188,11.499209,27.7228,2.652338,12.6733,29.5248
3,AAPL,133.72,N/A,41.11,73.6634,34.00,96.0396,8.2800,84.1584,29.332645,81.7822,21.066187,93.8614,85.901
4,ABBV,110.39,N/A,23.56,44.7525,13.00,85.1485,4.8900,68.7129,16.218431,47.9208,9.445878,63.3663,61.9802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.53,N/A,32.55,62.1782,-4.13,4.15842,5.9500,74.3564,23.323840,71.4851,9.889702,66.1386,55.6634
501,ZBH,165.08,N/A,-224.63,1.9802,2.87,38.5149,4.6800,63.5644,27.916793,79.2079,8.121371,50.8911,46.8317
502,ZBRA,421.33,N/A,46.98,77.8218,11.30,83.3663,5.2100,70.396,33.068671,84.3564,11.978332,74.4554,78.0792
503,ZION,49.80,N/A,20.80,40,1.16,11.5842,2.7300,44.4554,12.395598,32.8713,3.205843,17.2277,29.2277


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [ ]:
rv_dataframe.sort_values("RV Score", ascending=True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(inplace = True)
rv_dataframe.drop("index", axis = 1, inplace = True)
rv_dataframe

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [123]:
portfolio_input()

Enter the value of your portfolio:2500000


In [ ]:
position_size = portfolio_size / len(rv_dataframe.index )
for row in rv_dataframe.index:
  rv_dataframe.loc[row, "Number of Shares to Buy"] = math.floor(position_size / rv_dataframe.loc[row, "Price"])

rv_dataframe

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [157]:
writer = pd.ExcelWriter("value_strategy.xlsx", engine = "xlsxwriter")

rv_dataframe.to_excel(writer, sheet_name="Value Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [158]:


background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )



In [159]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [160]:
writer.save()